## Full doc (MR)

<span style="color: red;">

TODO:    
- SF
    - Check the three different virial parameters for star formation models...
- Feedback
    - Check equations for kinetic feedback. Yohan's code was CRAZY HARD to read. Why is his E_SN 10 times lower?
    - `ndebris`: obsolete parameter? Computed automatically as `ndebris_tot` and used nowhere!
- It would be nice to plot a RHD andAGN specific plot.
</span>

<span style="color: orange;">
For the detailed RAMSES doc maybe:    
<p>&nbsp;</p> <!-- Add vertical space -->

- The turbulent parameter 
    - ${b_{\text{turb}} = 1 + \left(\frac{1}{\text{ndim}} - 1\right) \zeta}$.

    - If $p_{\text{comp}}*N_\text{dim}-1 = 0$, $\zeta = 0.5$, else: 
    $\zeta = \frac{(p_{\text{comp}} - 1) + \sqrt{(p_{\text{comp}}^2 - p_{\text{comp}})(1 - N_\text{dim})}}{p_{\text{comp}} N_\text{dim} - 1}$   

    - $p_{\text{comp}}$ even more complicated. No clue what that is.
<p>&nbsp;</p> <!-- Add vertical space -->

- Turbulence:
    - Only used if `sf_virial=.true.` and a positive dissipation timescale `sf_tdiss` is set. Use `sf_compressive=.true.` (and add 1 to NPSCAL) to store and advect the compressive and solenoidal turbulent field in two different hydro variables.
    - Traced through `uold(_ ,ivirial1)` and `uold(_ ,ivirial2)` in the code.
    - Taken into account directly through $\sigma_V$ (sigma_2 in the code) for star formation. 

</span>

### Understanding the namelist
In the previous sections, we tested different simulation types without delving into the details of their parameters, i.e. the namelist `setup.nml`. In this section we explain in more details how the namelist works (see also [the documentation](https://ramses-organisation.readthedocs.io/en/latest/wiki/Runtime_Parameters.html) here).   

The namelist is split in several parameters section, all starting with the symbol `&`. For example, when we included radiative transfer (RT), we turned on radiative transfer with `rt=.true.` in the `&RUN_PARAMS` section, and set RT-specific params in the `&RT_PARAMS` section. You can access the list of all these parameters in their respective ramses modules, `ramses/amr/amr_parameters.f90` for `&RUN_PARAMS`, `ramses/rt/rt_parameters.f90` for `&RT_PARAMS`, and so on.

In the next two subsections, we look at key parameters from both `&SF_PARAMS` and `&FEEDBACK_PARAMS`.

## Star formation

In RAMSES, star formation parameters are set in the `&SF_PARAMS` section of the namelist. The full list of parameters available can be found in `ramses/hydro/read_hydro_parameters.f90`, and their default values are defined in `ramses/amr/amr_parameters.f90`. The details of the models presented below can be found in `ramses/pm/star_formation.f90`.   
Star formation in RAMSES occurs in three steps:
1. Restrict star formation to specific gas cells.
2. Define with what efficiency the gas mass will be converted into stars.
3. Compute the number of stellar particles that will be formed (i.e. particles representing simple stellar populations). 

In the current version of RAMSES, there are 7 star formation models, and more might be implemented later. They differ in the selection of star forming cells (step 1) and in the computation of the star formation efficiency $\epsilon_{ff}$ (SFE, step 2). For step 3, all star formation models are based on the Schmidt law to compute how many stars will be created, following
$$ \dot{\rho}_\star = \epsilon_{ff} \frac{\rho}{t_{ff}}, $$
with $\dot{\rho}_\star$ the rate of conversion of gas mass into stars, $\epsilon_{ff}$ the conversion efficiency, $\rho$ the density of the gas, and $t_{ff}=\left[3\pi / 32G\rho \right]^{1/2},$ with $G$ the gravitational constant. 

For the following section, we additionally define $n_\mathrm{H}$ the Hydrogen density (proportional to rho) and the Mach number, defined as ${\mathcal{M} = \frac{\sigma_V}{c_s}}$, with $\sigma_V$ the velocity dispersion and ${c_s^2 = (\gamma-1)\frac{P}{\rho} - T_{2\star}(n_\text{H}/n_\star)^{g_\star-1}}$ the sound speed of the medium (corrected for polytrope). Also, key namelist parameters for these models are listed in the table below.

<div align="center">

Namelist parameter | In equations     | Type          | Default | Description
------------------ | ---------------- | ------------- | ------- | ----------- 
`sf_virial`        |                  | Boolean       | false   | Enable turbulent star formation prescriptions
`sf_model`         |                  | Integer (0-5) | 3       | Turbulent star formation model (with `sf_virial=.true.`)
`eps_star`         | $\epsilon_\star$ | float         | 0       | Star formation efficiency
`n_star`           | $n_\star$        | float         | 0.1     | Star formation density threshold in H/cc
`m_star`           | $m_\star$        | float         | -1      | Star particle mass in units of mass_sph
`T2_star`          | $T_{2\star}$     | float         | 0       | Typical ISM polytropic temperature
`g_star`           | $g_\star$        | float         | 1.6     | Typical ISM polytropic index
</div>

If you use a model with `sf_virial=.true.`, subgrid turbulence will be modelled, so you will need one more hydro variable (i.e. add 1 to `NPSCAL` in the Makefile). Note that it only has an indirect effect on star formation with constant efficiency while it directly impacts the SFE through $\sigma_V$ for models with a variable $\epsilon_\star$.

In the next two subsections, we document the 7 models and split them in two parts, those with a constant efficiency, and those with a variable one. The first line after the model name shows the value of the namelist parameters to define in order to use a given model. 


### Star formation models (step 1 & 2)
#### Constant efficiency

Three of the models only change in the criterion to select star forming cells and use a constant star formation efficiency of $\epsilon_{ff} = \epsilon_\star$. We detail in the following list their respective condition for star formation.

- <span style="color: turquoise;">**Schmidt law**</span> ([Schmidt 1959](https://ui.adsabs.harvard.edu/abs/1959ApJ...129..243S))   

    `sf_virial=.true.` and `sf_model=0`
    
    > $n_\mathrm{H} > n_\star$
    <p>&nbsp;</p> <!-- Add vertical space -->

- <span style="color: turquoise;">**Schmidt law with polytrope**</span>

    `sf_virial=.false.`
    
    > $n_\mathrm{H} > n_\star$   
    > $T - \mathrm{T_{2\star}}\left(\frac{n_\mathrm{H}}{n_\star}\right)^{\mathrm{g_\star}-1} \leq 2\times10^4\rm\ K$   
    > $\rho>\mathrm{var\_cut\_refine}$   
    
    Where `var_cut_refine` is only used if a variable-based refinement is set and its role here is to restrict star formation in the refined region.
    <p>&nbsp;</p> <!-- Add vertical space -->


- <span style="color: turquoise;">**Virial criterion**</span>

    `sf_virial=.true.` and `sf_model=3`
    
    > $n_\mathrm{H} > n_\star$   
    > $\alpha_{0\rm,vir} = \frac{\sigma_V^2-c_s^2 \frac{\nabla^2\rho}{\rho}}{4 \pi G \rho\Delta x^2} < 0$ <span style="color: red;">TODO: cf below...</span>    
    > $\nabla^2\rho < 0$   <span style="color: red;">TODO: not really this, there are neighbouring cells in the computation...</span>
<p>&nbsp;</p> <!-- Add vertical space -->


- <span style="color: turquoise;">**Another virial criterion**</span> ([Hopkins 2013](https://ui.adsabs.harvard.edu/abs/2013MNRAS.432.2647H))

    `sf_virial=.true.` and `sf_model=5`
    
    > $n_\mathrm{H} > n_\star$  
    > $\alpha_{0\rm,hop} = \frac{(\nabla \vec{v})^2 + (\nabla \times \vec{v})^2}{2\rho G} < 1$ <span style="color: red;">TODO: not really v, there are neighbouring cells in the computation...</span>

    <!-- alpha0    = 0.5d0*(divv2+curlv2)/(factG*d)
    if(alpha0<1.0) then eps_star -->


#### Variable efficiency

The other three models all follow the same unique condition to allow star formation as $n_\mathrm{H} > \mathrm{n_\star}$. However, they use a varying star formation efficiency depending on the local conditions, and $\epsilon_\star$ now corresponds to a star formation efficiency at a much smaller scale. We detail in the following list the computation of their star formation efficiency.

- <span style="color: turquoise;">**A Simple Law of Star Formation**</span> ([Padoan 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...759L..27P))

    `sf_virial=.true.` and `sf_model=4`   
    $${\epsilon_{ff} = \epsilon_\star \exp\left(-1.6 \frac{t_{\text{ff}}}{t_{\text{dyn}}}\right),}$$
        
    with ${t_{\text{dyn}} = \frac{\Delta x}{2 \sqrt{\sigma_V^2 + c_s^2}}}$ and ${t_{\text{ff}} = 0.5427 \sqrt{\frac{1}{\rho G}}}$.
    <p>&nbsp;</p> <!-- Add vertical space -->
    

- <span style="color: turquoise;">**Multi-freefall models**</span> ([Federrath and Klessen 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...761..156F))   
The next two models are commonly referred to as multi-freefall or gravoturbulent models and share some common variables. These are namely the virial parameter ${\alpha_{0\rm,ff} = \frac{5 \sigma_V^2}{\pi \rho G \Delta x^2}}$ and the turbulent density fluctuations ${\sigma_s = \log\left(1.0 + b_{\text{turb}}^2 \mathcal{M}^2 \frac{\beta}{\beta + 1.0}\right)}$. $b_{\text{turb}}$ is the turbulent forcing parameter and $\beta = \frac{P_\text{th}}{P_\text{mag}}$ the ratio of thermal to magnetic pressure which tends to infinity when there is no magnetic field in the simulation. The two models available in RAMSES use the same equation to compute the star formation efficiency, which consists in integrating a log-normal density probability distribution function $p(s)$ ($s\equiv ln(\rho/\rho_0)$) weighted by the overdensity $\rho/\rho_0$ and a freefall time factor $t_{\mathrm{ff}}(\rho_0)/t_{\mathrm{ff}}(\rho)$ from a critical density $s_\text{crit}$ up to infinity: 
    $$ \epsilon_{ff} 
        = \frac{\epsilon}{\phi_t} \int_{s_{\mathrm{crit}}}^{\infty} \frac{t_{\mathrm{ff}}\left(\rho_0\right)}{t_{\mathrm{ff}}(\rho)} \frac{\rho}{\rho_0} p(s) ds
        = \epsilon_\star \frac{\phi_t}{2} \exp\left(\frac{3}{8} \sigma_s\right) \left[2 - \text{erf}\left(\frac{\sigma_s - s_\text{crit}}{\sqrt{2\sigma_s}}\right)\right], $$ 
    although with different definitions of $\phi_t$. We highlight their differences in the next two points.

    - <span style="color: turquoise;">**Multi-ff KM model**</span> ([Krumholz & McKee 2005](https://ui.adsabs.harvard.edu/abs/2005ApJ...630..250K/abstract))
    `sf_virial=.true.` and `sf_model=1` $${s_\mathrm{crit, KM} = \log\left[\frac{\alpha_{0\rm,ff}}{5} (\mathcal{M}\pi\phi_x)^2 \frac{1}{1+\beta^{-1}} \right]}, $$ and $\phi_t=0.49$ and $\phi_x=0.19$ with MHD ($\phi_t=0.46$ and $\phi_x=0.17$ without MHD).
    <p>&nbsp;</p> <!-- Add vertical space -->

    - <span style="color: turquoise;">**Multi-ff PN model**</span> ([Padoan & Nordlund 2012](https://ui.adsabs.harvard.edu/abs/2011ApJ...730...40P/abstract))
    `sf_virial=.true.` and `sf_model=2` $$ s_\text{crit, PN} = \log\left(\frac{0.067}{\theta^2} \alpha_{0\rm,ff} \mathcal{M}^2 f_{\beta}\right), $$ with ${f_{\beta} = \frac{\left(1+0.925 \beta^{-\frac{3}{2}}\right)^{\frac{2}{3}}}{\left(1+\frac{1}{\beta}\right)^2}}$ and $\phi_t=0.47$ and $\theta=1$ with MHD ($\phi_t=0.49$ and $\theta=0.97$ without MHD).


### Creating stellar particles (step 3)

In RAMSES, we use stellar particles to model simple stellar populations (stars born at the same time in the same medium). 

- The mass resolution: $M_\mathrm{\star, min}$   
    As for the RAMSES AMR grid, particles have a given resolution. For stellar particles, the resolution is based by default on the namelist parameter `m_star` unless it is set negative. The mass resolution is then given by $\rm M_\mathrm{\star, min} = m_\star m_\mathrm{sph}$. $m_\mathrm{sph}$ defines a typical mass scale in the simulation and need to be set by hand in idealised simulations through the namelist parameter `mass_sph` (0 by default), set in the section`&REFINE_PARAMS`. In cosmological simulations, $m_\mathrm{sph}$ is automatically set as $m_\mathrm{sph}=\frac{\Omega_b}{\Omega_m}0.5^{N_{dim}l_{max}}$, with $\Omega_b$ and $\Omega_m$ cosmological parameters, $N_{dim}$ the number of dimensions in the simulation, and $l_{max}$ the maximum level of the simulation. If `m_star` is set negative, then the mass of the stellar particle is set based on the star formation threshold `n_star`, as $\rm M_\mathrm{\star, min}=n_\star\Delta x^3$.

- Setting the mass of the stellar particle   
    All star particles formed have a mass given by an integer number of times the minimal mass of a stellar particle $M_\mathrm{\star, min}$, with a maximum of $0.9 m_{cell}$, with $m_{cell}$ the gas mass of the cell undergoing star formation. This number is randomly generated following a Poisson distribution with a mean of $\epsilon_{ff} \frac{m_{cell}}{M_\mathrm{\star, min}} \frac{dt}{t_\star}$, with $dt$ the current timestep, and $t_\star = 0.5427 \sqrt{\rho G}^{-1}$ the freefall time of an homogeneous sphere.

Once a star is created, it is characterised by an ID, its position, velocity, age, initial mass, and current mass (they can lose mass through supernovae explosions). If metals are modelled in the simulation, the star inherits the metallicity of the cell in which it formed.

## Stellar feedback

In RAMSES, stellar feedback parameters are set in the `&FEEDBACK_PARAMS` section of the namelist. The full list of parameters available can be found in `ramses/hydro/read_hydro_parameters.f90` and their default values are defined in `ramses/amr/amr_parameters.f90`. The details of the models presented below can be found in `ramses/pm/feedback.f90`.
When stars reach an age of `t_sne` Myr, they explode into supernovae and inject mass, energy, and metals if metal advection is enabled (i.e. if `metal=.true.`) in the surrounding media. The ejecta mass is usually set by `eta_sn` as $M_\text{ej} = \eta_{SN}M_\star$, with $M_\star$ the mass of the stellar particle. Similarly, if metal advection is enabled, the ejected mass has a metallicity of $Z_\text{ej} = Z_\star+\eta_Z(1-Z_\star)$ which corresponds to the metallicity of the star ($Z_\star$) added to its yield $\eta_Z$ (set by `yield`), i.e. the metal enrichment from the star. Although slight modifications can be made on these two quantities, the main differences between feedback models will be how the energy is injected, how much, and in which form. In its current version, RAMSES includes three models of stellar feedback along some possible additional alterations. 

<div align="center">

Namelist parameter | Type    | Default | Description
------------------ | ------- | ------- | -----------
`t_sne `           | float   | 10      | Supernova blast time in [Myr]
`eta_sn`           | float   | 0       | Supernova mass fraction
`yield`            | float   | 0       | Supernova metal yield
</div>


- <span style="color: turquoise;">**Thermal feedback**</span> 
    The first model implemented in RAMSES is thermal feedback, and injects energy in the cell hosting the stellar particle as $$\Delta E = M_\text{ej}(E_\text{kin} + E_\text{th}).$$ 
    In this equation, we include the kinetic energy $E_\text{kin}$ lost by the stellar particle as well as additional thermal energy injected by the supernova $E_\text{th}$. For this energy, we use a specific energy for type II SNe of $E_\text{SN}=10^{51}\rm erg / 100\, M_\odot$.

    Three additional alterations can be made to this model (they can all be used at the same time).  
    - <span style="color: turquoise;">**Resolved IMF modelling**</span>

        If `sf_imf=.true.`, the code will follow the initial mass function by assigning a single supernova ejected mass fraction `eta_sn` to each stellar particle with a mass between `mass_sne_min` and `mass_star_max`. Stellar particles less massive than those will have no ejected mass, while those more massive will have an ejected mass fraction given by `eta_sn`. This is usually used when the resolution is high enough to model stars individually.

    - <span style="color: turquoise;">**Stochastic GMC explosions**</span>

        Secondly, to account for stochastic GMC exploding, it is possible to increase the mass loss and energy injection by a factor $M_\text{GMC}/M_{\star,\text{min}}$, with $M_{\star,\text{min}}$ the minimal mass of stellar particles and $M_\text{GMC}$ set by the namelist parameter `mass_gmc`. As per its name, this boost is stochastically sampled and can only be positive.

    - <span style="color: turquoise;">**Infrared photon trapping**</span>

        The last alteration is the infrared photon trapping boost and is activated by setting the IR dust opacity for SN feedback `kappa_IR` ($\kappa_\text{IR}$). It only affects cells with a hydrogen density higher than $n_\text{H} = 10\rm\ H/cc$. This boost is given by $$E_\text{th} = E_\text{SN}\left[1+\frac{E_\text{RAD}}{E_\text{SN}} \left(1-\exp(-\kappa_\text{IR}\frac{Z_\text{gas}}{Z_\text{ave}} 200\rm\,pc)\right)\right],$$ 
        with $E_\text{RAD}=10^{53}$ the radiation specific energy for the lifetime of a <span style="color: red;">massive (right?)</span> star, $Z_\text{gas}$ the gas metallicity, $Z_\text{ave}$ the initial metallicity (set by `z_ave`), and $200\rm\,pc$ a fixed length over which the column density of the gas is estimated. Note that if metal advection is not enabled (i.e. if `metal=.false.`), $Z/Z_\text{ave}$ is replaced by $\rho/0  2$.

<div align="center">

Namelist parameter | Type    | Default | Description
------------------ | ------- | ------- | -----------
`sf_imf`           | boolean | false   | Model Initial Mass Function during thermal feedback events
\| `eta_ssn`       | float   | 0.95    | Single supernova ejected mass fraction
\| `mass_sne_min`  | float   | 10      | Min. stellar mass for SN [Msun]
\| `mass_star_max` | float   | 120     | Max. stellar mass for SN [Msun]
`mass_gmc`         | float   | 0       | Stochastic exploding GMC mass in solar mass
`kappa_IR`         | float   | 0       | IR dust opacity for supernova feedback
</div>
<p>&nbsp;</p> <!-- Add vertical space -->


- <span style="color: turquoise;">**Delayed cooling**</span> ([Teyssier et al. 2013](https://ui.adsabs.harvard.edu/abs/2013MNRAS.429.3068T))

    The second model available follows the same routine as thermal feedback (all the alterations thus work for it too) but aims to compensate the overcooling problem (spurious energy loss) by turning gas cooling off for a given period. This model is activated with by setting `delayed_cooling=.true.` and specifying a dissipation timescale `t_diss` (in $Myr$).

<div align="center">

Namelist parameter | Type    | Default | Description 
------------------ | ------- | ------- | -----------
`delayed_cooling`  | boolean | false   | Enable delayed cooling (need 1 more hydro variable)
\| `t_diss`        | float   | 20      | Dissipation timescale for SN feedback [Myr]
</div>
<p>&nbsp;</p> <!-- Add vertical space -->


- <span style="color: turquoise;">**Kinetic feedback**</span> ([Dubois & Teyssier (2008)](https://ui.adsabs.harvard.edu/abs/2008A%26A...477...79D))

    Another common version of SN feedback implemented in RAMSES is the kinetic feedback, triggered by setting a positive value for `f_w` and `eta_sn`. In this version, instead of injecting the SN energy as thermal energy in the surrounding cells, it is injected as kinetic energy. By setting `f_w`, the star formation model is also slightly modified in the sense that a fraction of the gas $f_w M_\star$ is additionally removed from the cell forming a stellar particle. This gas will directly be injected in the ejecta s.t. ${M_\text{ej} = (f_w+\eta_{SN})M_\star}$. If `metal=.true.`, the ejecta metallicity is also modified and given by ${Z_\text{ej} = Z_\star+\eta_Z(1-Z_\star)\frac{\eta_\text{SN}}{f_w+\eta_\text{SN}}}$. This ejecta is reinjected in cells within `rbubble` pc (with a minimum of $2\Delta x$) with a momentum of $$u_{\text{Sedov}} = \sqrt{f_{\text{ek}} \eta_{\text{sn}} E_{\text{SN}} \frac{M_\star}{M_{\text{ej}}}}.$$
    
    In this equation, ${E_\text{SN} = 10^{51}\rm\ erg/10\ M_\odot}$ and `f_ek` is the fraction of energy injected in the form of kinetic energy. Alongside kinetic energy, the remaining of the energy is injected as thermal energy: $$p_{\text{gas}} = (1 - f_{\text{ek}}) \eta_{\text{sn}} E_{\text{SN}} M_\star.$$
    The code is also adapted to allow <span style="color: turquoise;">resolved IMF modelling</span> (see section on the thermal feedback).

<div align="center">

Namelist parameter | Type    | Default | Description
------------------ | ------- |-------- |------------
`f_ek`             | float   | 0       | Supernova kinetic energy fraction (only between 0 and 1)
\|`f_w`            | float   | 0       | Supernova mass loading factor
\|`rbubble`        | float   | 0       | Supernova superbubble radius [pc]
</div>
<!-- \|`ndebris`      | integer | 1       | Supernova debris particle number -->
<p>&nbsp;</p> <!-- Add vertical space -->


- <span style="color: turquoise;">**Patches**</span>

    There are numerous additional modifications that can be made to model stellar feedback such as sampling SN explosions in time or injecting different momenta depending on how resolved the explosion site is. While not all of them are included in the core of the RAMSES code, some are available in the form of patches, such as `mom2` ([Kretschmer & Teyssier (2020)](https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.1385K)). Feel free to try them out!